In [200]:
def is_prime(n):
    if n <= 1:
        return False
    if n <= 3:
        return True
    if n % 2 == 0 or n % 3 == 0:
        return False
    i = 5
    while i * i <= n:
        if n % i == 0 or n % (i + 2) == 0:
            return False
        i += 6
    return True

In [223]:
def set_if_inside (m, x,y,value):
    rows, cols = m.shape
    if 0 <= x < rows and  0 <= y < cols:
        m[x, y] = value
        return True
    return False

In [233]:
def fill_primes(m):
    rows, cols = m.shape
    mid = round(rows * 0.5)
    side = 0
    i = start_text.value
    x = mid
    y = mid
    set_if_inside (m,x,y,2)
    while i < rows * cols:
        side += 1
        for j in range (side):
            x += 1
            if is_prime(i):
                set_if_inside(m, x, y, 1)
            i += 1
        for j in range (side):
            y += 1
            if is_prime(i):
                set_if_inside(m, x, y, 1)
            i += 1
        side += 1
        for j in range (side):
            x -= 1
            if is_prime(i):
                set_if_inside(m, x, y, 1)
            i += 1
        for j in range (side):
            y -= 1
            if is_prime(i):
                set_if_inside(m, x, y, 1)
            i += 1
    

In [236]:
def display_primes():
    size = round(limit_text.value**0.5)
    matrix = np.zeros((size+10,size+10))
    fill_primes(matrix)
    ys, xs = np.where(matrix == 1)
    y0, x0 = np.where(matrix == 2)
    
    with output:
        clear_output(wait=True)
        plt.close('all')  
        plt.figure(figsize=(15/1000*size,15/1000*size))
        plt.scatter(x0, y0, c='red', s=50)
        plt.scatter(xs, ys, c='blue', s=1)
        plt.xlim(0,size)
        plt.ylim(0,size)
        plt.grid(False)
        plt.show()

In [237]:
import ipywidgets as widgets
from IPython.display import display, clear_output
import numpy as np
import matplotlib.pyplot as plt

initial_limit = 10000
initial_start = 1

output = widgets.Output()


# --- Data and initial plot ---
size = round(initial_limit ** 0.5)
matrix = np.zeros((size,size))

# --- Widgets ---
limit_text = widgets.IntText(
    value=initial_limit,
    description='Checks primes up to:',
    placeholder='Enter a number',
)

start_text = widgets.IntText(
    value=initial_start,
    description='Starting from:',
    placeholder='Enter a number',
)


button = widgets.Button(
    description="Display primes",
    button_style="info",  # optional (success, info, warning, danger)
    tooltip='Use the number in the text box',
)

# --- Callback: update the existing ---
def on_button_clicked(Button):
    try:
        size = round(limit_text.value**0.5)
        display_primes()
    except ValueError:
        size = round (initial_limit ** 0.5)
        print("Invalid input, using size " )


# Bind the function to the button's click event
button.on_click(on_button_clicked)


# --- Show UI ---
row1 = widgets.HBox([limit_text, start_text])
row2 = widgets.HBox([button])
display(widgets.VBox([row1, row2, output]))

